### Imports

In [1]:
import sys 
sys.path.append('C:\\Users\\Eesh Gupta\\Documents\\RU Research\\Chakram')

In [2]:
from ECD import BatchOptimizer as BO_ECD
from DECD_Old import BatchOptimizer as BO_DECD


Need tf version 2.3.0 or later. Using tensorflow version: 2.7.0



In [3]:
#%%
# note: timestamp can't use "/" character for h5 saving.
TIMESTAMP_FORMAT = "%Y-%m-%d %H:%M:%S"
END_OPT_STRING = "\n" + "=" * 60 + "\n"
import numpy as np
import tensorflow as tf

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)  # supress warnings
import h5py

# print(
#     "\nNeed tf version 2.3.0 or later. Using tensorflow version: "
#     + tf.__version__
#     + "\n"
# )
import ECD_control.ECD_optimization.tf_quantum as tfq
from ECD_control.ECD_optimization.visualization import VisualizationMixin
import qutip as qt
import datetime
import time

### Single ECD g0 ->g1

Initial Testing Code

In [4]:
#The target oscillator state.
N =10
psi_i = qt.basis(N, 0) # initial state
psi_t = qt.basis(N,1) #target state

#Opt Params
#Optimization of ECD Circuit parameters (betas, phis, and thetas)
#the optimization options
ecd_opt_params = {
'N_blocks' : 5, #circuit depth
'N_multistart' : 10, #Batch size (number of circuit optimizations to run in parallel)
'epochs' : 1000, #number of epochs before termination
'epoch_size' : 10, #number of adam steps per epoch
'learning_rate' : 0.01, #adam learning rate
'term_fid' : 0.9999, #terminal fidelitiy
'dfid_stop' : 1e-6, #stop if dfid between two epochs is smaller than this number
'beta_scale' : 3.0, #maximum |beta| for random initialization
'initial_states' : [qt.tensor(qt.basis(2,0),psi_i)], #qubit tensor oscillator, start in |g> |0>
'target_states' : [qt.tensor(qt.basis(2,0), psi_t)], #end in |e> |target>.
'name' : 'Fock %d' % 1, #name for printing and saving
'filename' : None, #if no filename specified, results will be saved in this folder under 'name.h5'
}


#note: optimizer includes pi pulse in every ECD step. However, final ECD step is implemented 
#in experiment as a displacement since the qubit and oscillator should be disentangled at this point.
#So, we ask the optimizer to end in |e> |target> instead of |g>|target>.

In [5]:
#Optimizer 
ecd_opt = BO_ECD(**ecd_opt_params)
ecd_opt.print_info()

optimization_type: state transfer
N_multistart: 10
N_blocks: 5
term_fid: 0.9999
dfid_stop: 1e-06
no_CD_end: False
learning_rate: 0.01
epoch_size: 10
epochs: 1000
beta_scale: 3.0
alpha_scale: 1.0
theta_scale: 3.141592653589793
use_displacements: False
use_phase: False
name: Fock 1
comment: 
N_cav: 10
filename: Fock 1.h5

Best circuit parameters found:
betas:         [-0.08254-0.33114j -0.93251-0.77643j -0.28432-1.47222j -0.29166+0.95669j
 -2.07364-1.06129j]
alphas:        [0.+0.j]
phis (deg):    [  0.      -52.4231  138.08344 -59.02718  22.70717]
thetas (deg):  [ -93.94506 -146.12735  -85.93204 -100.17455  145.30782]
Max Fidelity:  0.223138




In [6]:
ecd_opt.optimize()

Start time: 2022-05-22 03:52:53
 Epoch: 90 / 1000 Max Fid: 0.998170 Avg Fid: 0.955409 Max dFid: 0.000001 Avg dFid: 0.000000 Elapsed time: 0:00:22.473273 Remaing time: 0:03:47.2297611
 max dFid: 0.000001
dFid stop: 0.000001


 Optimization stopped.  No dfid is greater than dfid_stop

optimization_type: state transfer
N_multistart: 10
N_blocks: 5
term_fid: 0.9999
dfid_stop: 1e-06
no_CD_end: False
learning_rate: 0.01
epoch_size: 10
epochs: 1000
beta_scale: 3.0
alpha_scale: 1.0
theta_scale: 3.141592653589793
use_displacements: False
use_phase: False
name: Fock 1
comment: 
N_cav: 10
filename: Fock 1.h5

Best circuit parameters found:
betas:         [-0.40899-1.5132j  -1.13592+0.39294j  0.24807-0.65493j -0.08444-1.19726j
 -0.00001-0.00002j]
alphas:        [0.+0.j]
phis (deg):    [  0.       89.99414  89.98286  90.01535 179.98871]
thetas (deg):  [  90.00426   86.32995 -115.72317  -36.10104   89.99819]
Max Fidelity:  0.998170


all data saved as: Fock 1.h5
termination reason: dfid
optimization

'2022-05-22 03:52:53'

In [7]:
ecd_opt.best_circuit()

{'fidelity': 0.9981705,
 'betas': array([-4.0898558e-01-1.5132022e+00j, -1.1359162e+00+3.9293993e-01j,
         2.4806748e-01-6.5493113e-01j, -8.4440157e-02-1.1972619e+00j,
        -5.9318804e-06-2.2492326e-05j], dtype=complex64),
 'alphas': array([0.+0.j], dtype=complex64),
 'phis': array([0.       , 1.5706942, 1.5704973, 1.5710642, 3.1413958],
       dtype=float32),
 'thetas': array([ 1.5708706 ,  1.5067418 , -2.0197504 , -0.63008213,  1.5707648 ],
       dtype=float32)}

In [10]:
# saving angles 
betas = ecd_opt.best_circuit()['betas']
#gammas = opt.best_circuit()['gammas']
phis = ecd_opt.best_circuit()['phis']
thetas = ecd_opt.best_circuit()['thetas']
#etas = opt.best_circuit()['etas']
params = [np.real(betas), np.imag(betas), phis, thetas]
for i in range(len(params)):
    params[i] = [float(k) for k in params[i]]

fname = 'Test_g0_g1_May22.txt'
a_file = open(fname, "w")
np.savetxt(a_file, params)
a_file.close()

### Feeding ECD params to DECD

#### Convert ECD angles to DECD angles
Add gamma

In [44]:
#obtain ECD params
circ = ecd_opt.best_circuit()
betas = circ['betas']
alphas = circ['alphas']
phis = circ['phis']
thetas = circ['thetas']

#gammas (displacement of second mode)
gammas = np.copy(betas)
for i in range(len(gammas)):
    gammas[i] = 0.0 +0.0j

#etas (just ignore this)
etas = np.copy(betas)
for i in range(len(etas)):
    etas[i] = np.pi/2 #pi/2

#new initial params (Needs 2 alphas for mode 1 and mode 2)
init_params = [betas, gammas, alphas, alphas, phis, etas, thetas]

In [45]:
etas

array([1.5707964+0.j, 1.5707964+0.j, 1.5707964+0.j, 1.5707964+0.j,
       1.5707964+0.j], dtype=complex64)

In [46]:
#Since DECD has more than 1 multistart, we will make sure all multistarts
# have the same initial params-- namely the ecd onces
N_multistart = 10
init_params_more_multi = [np.array([i for j in range(N_multistart)]) for i in init_params]

In [47]:
init_params_more_multi

[array([[-2.0400467e+00+7.3097885e-01j, -6.0226512e-01+5.0592250e-01j,
         -7.5922124e-02-1.1985768e+00j,  6.3371098e-01-1.2616897e-01j,
         -1.0394355e-05-2.6693851e-05j],
        [-2.0400467e+00+7.3097885e-01j, -6.0226512e-01+5.0592250e-01j,
         -7.5922124e-02-1.1985768e+00j,  6.3371098e-01-1.2616897e-01j,
         -1.0394355e-05-2.6693851e-05j],
        [-2.0400467e+00+7.3097885e-01j, -6.0226512e-01+5.0592250e-01j,
         -7.5922124e-02-1.1985768e+00j,  6.3371098e-01-1.2616897e-01j,
         -1.0394355e-05-2.6693851e-05j],
        [-2.0400467e+00+7.3097885e-01j, -6.0226512e-01+5.0592250e-01j,
         -7.5922124e-02-1.1985768e+00j,  6.3371098e-01-1.2616897e-01j,
         -1.0394355e-05-2.6693851e-05j],
        [-2.0400467e+00+7.3097885e-01j, -6.0226512e-01+5.0592250e-01j,
         -7.5922124e-02-1.1985768e+00j,  6.3371098e-01-1.2616897e-01j,
         -1.0394355e-05-2.6693851e-05j],
        [-2.0400467e+00+7.3097885e-01j, -6.0226512e-01+5.0592250e-01j,
         -7.59

#### Initial Testing Code

In [48]:
#The target oscillator state.
N1 =10
N2 =10
Fock1 = 0
Fock2= 0
psi_i1 = qt.basis(N1,Fock1) #target state
psi_i2 = qt.basis(N2,Fock2)
psi_initial = qt.tensor(psi_i1, psi_i2)

In [49]:
Fock1 = 1
Fock2= 0
psi_t1 = qt.basis(N1,Fock1) #target state
psi_t2 = qt.basis(N2,Fock2)
psi_target = qt.tensor(psi_t1, psi_t2)
psi_target

Quantum object: dims = [[10, 10], [1, 1]], shape = (100, 1), type = ket
Qobj data =
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]

In [71]:
#Optimization of ECD Circuit parameters (betas, phis, and thetas)
#the optimization options
decd_opt_params = {
'N_blocks' : 5, #circuit depth
'N_multistart' : 1, #Batch size (number of circuit optimizations to run in parallel)
'epochs' : 100, #number of epochs before termination
'epoch_size' : 20, #number of adam steps per epoch
'learning_rate' : 0.1, #adam learning rate
'term_fid' : 0.05, #terminal fidelitiy
'dfid_stop' : 1e-6, #stop if dfid between two epochs is smaller than this number
'beta_scale' : 3.0, #maximum |beta| for random initialization
'gamma_scale' : 3.0, #maximum |gamma| for random initialization
'N_cav1': N1, #number of levels in mode 1
'N_cav2': N2, #number of levels in mode 2
'initial_states' : [qt.tensor(qt.basis(2,0),psi_initial)], #qubit tensor oscillator, start in |g> |0>
'target_states' : [qt.tensor(qt.basis(2,0), psi_target)], #end in |e> |target>.
#"initial_params": init_params,
'name' : 'Fock1 %d' % Fock1, #name for printing and saving
'filename' : None, #if no filename specified, results will be saved in this folder under 'name.h5'
}



#### Running DECD

In [72]:
decd_opt = BO_DECD(**decd_opt_params)
tf.config.run_functions_eagerly(True) # for some reason, important
decd_opt.print_info()

None
optimization_type: state transfer
N_multistart: 1
N_blocks: 5
term_fid: 0.05
dfid_stop: 1e-06
no_CD_end: False
learning_rate: 0.1
epoch_size: 20
epochs: 100
beta_scale: 3.0
gamma_scale: 3.0
alpha1_scale: 1.0
alpha2_scale: 1.0
theta_scale: 3.141592653589793
use_etas: False
use_displacements: False
use_phase: False
name: Fock1 1
comment: 
initial_params: None
N_cav1: 10
N_cav2: 10
filename: Fock1 1.h5

Best circuit parameters found:
betas:         [ 0.03567-0.06805j  0.08509-0.91597j  0.73314-0.30746j -0.84777-1.79782j
  0.34213+0.11275j]
gammas:         [ 0.65304-2.04151j -1.05401-0.64587j -0.46462+0.30259j -0.92531+0.931j
 -0.33601-0.33607j]
alphas1:        [0.+0.j]
alphas2:        [0.+0.j]
phis (deg):    [   0.       -98.34    -167.85622   18.33297  -80.76675]
etas (deg):    [89.99999 89.99999 89.99999 89.99999 89.99999]
thetas (deg):  [   8.10717 -170.1581    57.02086  -67.22957  -99.31422]
Max Fidelity:  0.003517




In [73]:
decd_opt.optimize()

Start time: 2022-05-20 05:40:27
 Epoch: 1 / 100 Max Fid: 0.101645 Avg Fid: 0.101645 Max dFid: 0.098128 Avg dFid: 0.098128 Elapsed time: 0:00:01.537549 Remaing time: 0:02:32.217353

 Optimization stopped. Term fidelity reached.

optimization_type: state transfer
N_multistart: 1
N_blocks: 5
term_fid: 0.05
dfid_stop: 1e-06
no_CD_end: False
learning_rate: 0.1
epoch_size: 20
epochs: 100
beta_scale: 3.0
gamma_scale: 3.0
alpha1_scale: 1.0
alpha2_scale: 1.0
theta_scale: 3.141592653589793
use_etas: False
use_displacements: False
use_phase: False
name: Fock1 1
comment: 
initial_params: None
N_cav1: 10
N_cav2: 10
filename: Fock1 1.h5

Best circuit parameters found:
betas:         [ 0.03567-0.06805j  0.08509-0.91597j  0.73314-0.30746j -0.84777-1.79782j
  0.34213+0.11275j]
gammas:         [ 0.65304-2.04151j -1.05401-0.64587j -0.46462+0.30259j -0.92531+0.931j
 -0.33601-0.33607j]
alphas1:        [0.+0.j]
alphas2:        [0.+0.j]
phis (deg):    [   0.       160.06102 -115.14635   69.35605  -11.13934]


'2022-05-20 05:40:27'

In [35]:
decd_opt.batch_fidelities(decd_opt.betas_rho,
                decd_opt.betas_angle,
                decd_opt.gammas_rho,
                decd_opt.gammas_angle,
                decd_opt.alphas1_rho,
                decd_opt.alphas1_angle,
                decd_opt.alphas2_rho,
                decd_opt.alphas2_angle,
                decd_opt.phis,
                decd_opt.etas,
                decd_opt.thetas,)

<tf.Tensor: shape=(), dtype=float32, numpy=8.0747334e-13>

#### Inspecting Gradients near minima

In [74]:
@tf.function
def loss_fun(fids):
    #print('loss fun called')
    # I think it's important that the log is taken before the avg
    losses = tf.math.log(1 - fids)
    avg_loss = tf.reduce_sum(losses) / decd_opt.parameters["N_multistart"]
    return avg_loss

In [75]:
variables = [decd_opt.betas_rho,
            decd_opt.betas_angle,
            decd_opt.gammas_rho,
            decd_opt.gammas_angle,
            decd_opt.phis,
            decd_opt.thetas]

with tf.GradientTape() as tape:
    fid = decd_opt.batch_fidelities(decd_opt.betas_rho,
            decd_opt.betas_angle,
            decd_opt.gammas_rho,
            decd_opt.gammas_angle,
            decd_opt.alphas1_rho,
            decd_opt.alphas1_angle,
            decd_opt.alphas2_rho,
            decd_opt.alphas2_angle,
            decd_opt.phis,
            decd_opt.etas,
            decd_opt.thetas,)
    new_loss = loss_fun(fid)
    #tape = tf.GradientTape()
    dloss_dvar = tape.gradient(new_loss, variables)
    print(dloss_dvar)
                   

[None, None, None, None, <tf.Tensor: shape=(5, 1), dtype=float32, numpy=
array([[ 0.00117935],
       [-0.02273876],
       [-0.00517935],
       [ 0.00023586],
       [ 0.00974743]], dtype=float32)>, <tf.Tensor: shape=(5, 1), dtype=float32, numpy=
array([[-0.01013791],
       [-0.02023907],
       [-0.02267454],
       [-0.02169513],
       [ 0.00093247]], dtype=float32)>]


In [70]:
fid

<tf.Tensor: shape=(), dtype=float32, numpy=0.9899562>

In [65]:
@tf.function
def loss_fun(fids):
    #print('loss fun called')
    # I think it's important that the log is taken before the avg
    losses = tf.math.log(1 - fids)
    avg_loss = tf.reduce_sum(losses) / ecd_opt.parameters["N_multistart"]
    return avg_loss

In [66]:
variables = [ecd_opt.betas_rho,
            ecd_opt.betas_angle,
            ecd_opt.phis,
            ecd_opt.thetas]

with tf.GradientTape() as tape:
    fid = ecd_opt.batch_fidelities(ecd_opt.betas_rho,
            ecd_opt.betas_angle,
            ecd_opt.alphas_rho,
            ecd_opt.alphas_angle,
            ecd_opt.phis,
            ecd_opt.thetas,)
    new_loss = loss_fun(fid)
    #tape = tf.GradientTape()
    dloss_dvar = tape.gradient(new_loss, variables)
    print(dloss_dvar)
                   

[<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[ 3.5368568e-05,  1.4582398e-06, -1.6648934e-04,  5.4512825e-04,
        -1.3194750e-08, -6.6370852e-03,  6.2279825e-05,  1.4816676e-05,
         4.1190069e-05,  2.0490918e-06],
       [-1.5194964e-04,  1.5983483e-07, -1.3974009e-03, -3.0532166e-02,
        -7.2742793e-08,  2.5009643e-02, -2.6463054e-04,  1.6805268e-04,
        -1.3527540e-02,  4.4460765e-05],
       [ 2.1258564e-04, -1.5518254e-06,  1.1042421e-03,  4.2838037e-02,
         2.4293021e-07, -2.7163293e-02,  1.2391097e-04, -1.1924920e-03,
         1.8107558e-02,  6.8114027e-06],
       [ 2.9340164e-05, -7.1960562e-06, -6.4653985e-04,  1.6693925e-02,
        -5.0462859e-07,  6.7628153e-02, -3.2525433e-05, -1.2029792e-04,
        -3.5097595e-02,  1.3094203e-05],
       [ 4.7652480e-05,  1.4139036e-06, -5.0548260e-04, -4.2997161e-03,
         4.1585000e-07,  4.2433850e-03,  6.5994158e-05, -3.6126067e-04,
         3.3981081e-02,  7.1213844e-06]], dtype=float32)>, <tf.Ten

In [67]:
fid

<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([0.9150901 , 0.8617574 , 0.8677291 , 0.91820776, 0.8480251 ,
       0.98889935, 0.99238324, 0.9129576 , 0.9898716 , 0.8917448 ],
      dtype=float32)>